# Example: Writing a power grid model to the extended 3DCityDB

This example shows how to write a power grid model to the extended *3DCityDB*, using the `dblayer` package. Basic knowledge about [CityGML](http://www.citygmlwiki.org) and the [Utility Network ADE](https://en.wiki.utilitynetworks.sig3d.org) are required for this notebook. A nice overview of the of the basic concepts of the Utility Network ADE can be found [here](https://en.wiki.utilitynetworks.sig3d.org/images/upload/20160913_-_Kolbe_-_Modeling_3D_Utility_Networks_%2B_UtilityNetwork_ADE_Core_Model.pdf).

For the example, a [CIGRE network](http://www.cigre.org/News/Network-of-the-Future) is used, as shown in the following figure.
<img src="./figures/cigre_network_lv.png" style="width: 650px;">

## Getting started ...

Import the required funtionality from package `dblayer`.

In [1]:
from dblayer.helpers.utn.electrical_network import *

Load data from CSV files, containing information about busses, lines and loads.

In [2]:
import pandas as pd
import os.path

busses_csv_data = pd.read_csv( os.path.join( 'csv', 'bus_data.csv' ), index_col = 0 )
lines_csv_data = pd.read_csv( os.path.join( 'csv', 'line_data.csv' ), index_col = 0 )
loads_csv_data = pd.read_csv( os.path.join( 'csv', 'load_data.csv' ), index_col = 0 )

## Connect to database

Define connection parameters for 3DCityDB instance.

In [3]:
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'testdb'
    )

Connect to database using an instance of class `DBAccess`.

In [4]:
db_access = DBAccess()
db_access.connect_to_citydb( connect )

For the purpose of this notebook yoi might want to use an empty 3DCityDB. To this end, the next two lines erase all data from an existing database. **Only execute the next two lines if you really want to erase all data from your database!**

In [5]:
db_access.cleanup_citydb_schema()
db_access.cleanup_simpkg_schema()

## Write the network to the database
Define the spatial reference ID (SRID) of the network. This should be consistent with the setup of your database.

In [6]:
srid = 25833

Insert a new *network* and an associated *network graph*. The *network* basically contains the topographical representation - the so-called *network features* - of all the network network components. The *network graph* contains the toplogical representation - of all these components the topological links between them.

In [7]:
ntw_id = db_access.add_citydb_object(
    insert_network,
    name = 'CIGRE_example_network',
    id = 1000
    )

ntw_graph_id = db_access.add_citydb_object(
    insert_network_graph,
    name = 'CIGRE_example_network_graph',
    network_id = ntw_id
    )

Add all busses to the network. Use function `write_bus_to_db` from package `dblayer`.

In [8]:
bus_data = {}

for index, bus_csv_data in busses_csv_data.iterrows():

    bus_data[bus_csv_data.name] = write_bus_to_db( 
        db_access,
        bus_csv_data.name,
        bus_csv_data.type,
        Point2D( bus_csv_data.x, bus_csv_data.y ),
        bus_csv_data.vn_kv,
        srid,
        ntw_id,
        ntw_graph_id
    )

Add all lines to the network and connect them with the busses. Use function `write_line_to_db` from package `dblayer`.

In [9]:
for index, line_csv_data in lines_csv_data.iterrows():

    write_line_to_db(
        db_access,
        line_csv_data.name,
        bus_data[line_csv_data.from_bus],
        bus_data[line_csv_data.to_bus],
        line_csv_data.c_nf_per_km,
        line_csv_data.r_ohm_per_km,
        line_csv_data.x_ohm_per_km,
        line_csv_data.max_i_ka,
        line_csv_data.type,
        srid,
        ntw_id,
        ntw_graph_id
    )

Add all transformers to the network and connect them with the busses. Use function `write_transformer_to_db` from package `dblayer`.

In [10]:
write_transformer_to_db(
    db_access,
    'tr-R0-R1',
    bus_data['bus-R0-MV'],
    bus_data['bus-R1-LV'],
    '0.63 MVA 20/0.4 kV',
    srid,
    ntw_id, ntw_graph_id
)

write_transformer_to_db(
    db_access,
    'tr-I0-I1',
    bus_data['bus-I0-MV'],
    bus_data['bus-I1-LV'],
    '0.25 MVA 20/0.4 kV',
    srid,
    ntw_id,
    ntw_graph_id
)

write_transformer_to_db(
    db_access,
    'tr-C0-C1',
    bus_data['bus-C0-MV'],
    bus_data['bus-C1-LV'],
    '0.4 MVA 20/0.4 kV',
    srid,
    ntw_id, ntw_graph_id
)

Add all switches to the network and connect them with the busses. Use function `write_switch_to_db` from package `dblayer`.

In [11]:
write_switch_to_db(
    db_access,
    'switch-S1',
    bus_data['bus-0-MV'],
    bus_data['bus-R0-MV'],
    'CB',
    srid,
    ntw_id, ntw_graph_id
)

write_switch_to_db(
    db_access,
    'switch-S2',
    bus_data['bus-0-MV'],
    bus_data['bus-I0-MV'],
    'CB',
    srid,
    ntw_id, ntw_graph_id
)

write_switch_to_db(
    db_access,
    'switch-S3',
    bus_data['bus-0-MV'],
    bus_data['bus-C0-MV'],
    'CB',
    srid,
    ntw_id,
    ntw_graph_id
)

Add the external grid as terminal element. Use function `write_terminal_element_to_db` from package `dblayer`.

In [12]:
write_terminal_element_to_db(
    db_access,
    'feeder',
    'external-grid',
    bus_data['bus-0-MV'],
    srid,
    ntw_id,
    ntw_graph_id
)

Add the loads to the network. Use function `write_load_to_db` from package `dblayer`.

In [13]:
for index, load_csv_data in loads_csv_data.iterrows():

    write_load_to_db(
        db_access,
        load_csv_data.name,
        bus_data[load_csv_data.bus],
        load_csv_data.p_kw,
        load_csv_data.q_kvar,
        srid,
        ntw_id,
        ntw_graph_id
    )

Commit changes to database.

In [14]:
db_access.commit_citydb_session()

Done.